In [12]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json, os

from pathlib import Path 

from distribution_substation_liang2022.v1.scripts import utils_sub # available at github.com/jieunbyun/network-datasets 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))

row_names = list(edges.keys()) + ['sys']

n_comp = len(edges)
n_var = n_comp + 1  # components + system
n_state = 2 # binary states: working (1) or failed (0)


In [14]:
def sfun_subs(comps_st, thres=360):
    node_groups = utils_sub.process_nodes(nodes)
    capacity = utils_sub.sys_fun(comps_st, edges, nodes, node_groups, return_details=False)

    if capacity >= thres:
        return capacity, 's', None
    else:
        return capacity, 'f', None

In [15]:
import importlib
importlib.reload(utils_sub)

<module 'distribution_substation_liang2022.v1.scripts.utils_sub' from 'C:\\Users\\jb622s\\git\\network-datasets\\distribution_substation_liang2022\\v1\\scripts\\utils_sub.py'>

# Find rules over thresholds

In [ ]:
pga_str_list = ['0_1g', '0_2g', '0_3g', '0_4g', '0_5g']
for pga_str in pga_str_list:
    print(f"--- {pga_str} ---")
    with open(rf"data\probs_{pga_str}.json", "r") as f: 
        probs_dict = json.load(f)
    
    probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names[:-1]]
    probs = torch.tensor(probs, dtype=torch.float32, device=device)

    # Threshold values to start with
    if os.path.exists(r"tsum_res/thres_list.json"):
        with open(r"tsum_res/thres_list.json", "r") as f:
            thres_list = json.load(f)
            thres_isdone = [False]*len(thres_list)

            if 0 in thres_list: 
                idx_0 = thres_list.index(0)
                thres_isdone[idx_0] = True # 0 threshold doesn't need to be computed
    else:
        thres_list = [360]
        thres_isdone = [False]
        with open(r"tsum_res/thres_list.json", "w") as f:
            json.dump(thres_list, f, indent=4)

    while not all(thres_isdone):

        # get the next highest threshold to compute
        i, t = next((i, t) for i, (t, done) in enumerate(zip(thres_list, thres_isdone)) if not done)
        thres_isdone[i] = True
        print(f"Threshold: {t}..")

        # initialisation
        sfun = lambda comps_st: sfun_subs(comps_st, thres=t)

        # Load previous rules if available from previous runs with other pga values
        if os.path.exists(Path("tsum_res") / f"rules_surv_{t}.json"):
            rules_surv = json.loads((Path("tsum_res") / f"rules_surv_{t}.json").read_text(encoding="utf-8"))
            rules_fail = json.loads((Path("tsum_res") / f"rules_fail_{t}.json").read_text(encoding="utf-8"))
            rules_mat_surv = torch.load(Path("tsum_res") / f"rules_surv_{t}.pt", map_location="cpu")
            rules_mat_surv = rules_mat_surv.to(device)
            rules_mat_fail = torch.load(Path("tsum_res") / f"rules_fail_{t}.pt", map_location="cpu")
            rules_mat_fail = rules_mat_fail.to(device)
        else:
            rules_surv = []
            rules_fail = []
            rules_mat_surv = None
            rules_mat_fail = None
            
        # find rules
        result = tsum.run_rule_extraction(
            sfun = sfun,
            probs = probs,
            row_names = row_names,
            n_state=n_state,
            rules_surv = rules_surv,
            rules_fail = rules_fail,
            rules_mat_surv = rules_mat_surv,
            rules_mat_fail = rules_mat_fail,
            output_dir = "tsum_res",
            surv_json_name = f"rules_surv_{t}.json",
            fail_json_name = f"rules_fail_{t}.json",
            surv_pt_name = f"rules_surv_{t}.pt",
            fail_pt_name = f"rules_fail_{t}.pt",
            metrics_path = f"metrics_{t}.jsonl",
            unk_prob_thres = 1e-6,
        )

        for s in result['sys_vals']:
            s=int(s)
            if s not in thres_list:
                thres_list.append(s)
                if s == 0:
                    thres_isdone.append(True) # no need to work on threshold 0
                else:
                    thres_isdone.append(False)
                # Sort the two lists by descending order of thres_list
                pairs = list(zip(thres_list, thres_isdone))
                pairs.sort(key=lambda x: x[0], reverse=True)
                thres_list, thres_isdone = map(list, zip(*pairs))
        print(f"Updated thres_list: {thres_list}")
        with open(r"tsum_res/thres_list.json", "w") as f:
            json.dump(thres_list, f, indent=4)

--- 0_1g ---
Threshold: 360..
---
Round: 1, Unk. prob.: 1.000e+00
No. of non-dominant rules: 20, Survival rules: 6, Failure rules: 14


C:\Users\jb622s\AppData\Local\Temp\ipykernel_40684\1737172208.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rules_mat_surv = torch.load(Path("tsum_res") / f"rules_sur

Failure sample found from survival rules 👍👍
No. of existing rules removed:  0
New rule added. System state: f, System value: 348.0. Total samples: 1024.
New rule (No. of conditions: 2): {'M_DS110_17': ('<=', 0), 'M_DS110_29': ('<=', 0), 'sys': ('<=', 0)}
Updated sys_vals: [348.0]
Failure sample found from failure rules 👍
No. of existing rules removed:  0
New rule added. System state: f, System value: 0.0. Total samples: 1024.
New rule (No. of conditions: 2): {'M_DS110_18': ('<=', 0), 'M_DS110_30': ('<=', 0), 'sys': ('<=', 0)}
Unique system values: [0.0, 348.0]
---
Round: 2, Unk. prob.: 1.000e+00
No. of non-dominant rules: 22, Survival rules: 6, Failure rules: 16
Survival sample found from survival rules 👍
No. of existing rules removed:  0
New rule added. System state: s, System value: 360.0. Total samples: 1024.
New rule (No. of conditions: 46): {'M_Line220_1': ('>=', 1), 'M_Line220_2': ('>=', 1), 'M_Line220_3': ('>=', 1), 'M_Line220_4': ('>=', 1), 'M_Line220_5': ('>=', 1), 'M_Line220_